In [141]:
from bs4 import BeautifulSoup
import requests
#USE PYTHON ANACADONA 3.8.5 to RUN


In [142]:
url= "http://onlinecollegebasketball.org/game/840273"
page = requests.get(url)
soup = BeautifulSoup(page.text,"html")

In [143]:
infoList = soup.find_all("td",class_="left")
gameData = {
  "awayTeam":{
    "name": infoList[1].text.replace("\xa0"," "),
    "teamCode":(infoList[1].find("a").get("href")).split("/")[2],
    "players":[] 
  }
  ,
  "homeTeam":{
    "name": infoList[2].text.replace("\xa0"," "),
    "teamCode":(infoList[2].find("a").get("href")).split("/")[2],
    "players":[]
  }
}



{'awayTeam': {'name': 'UH-Hilo Vulcans', 'teamCode': '565', 'players': []},
 'homeTeam': {'name': 'Dominican Univ. of California Penguins',
  'teamCode': '564',
  'players': []}}

In [144]:
teamSwitch = 0 
curTeam = "awayTeam"
infoListPlayers = infoList[4:]  # cuts it to the first player in BoxScore

for i in range(len(infoListPlayers)):
    if len(infoListPlayers[i].text.split("\xa0")) == 3:
        name, position = infoListPlayers[i].text.split("\xa0")[1:]
        
        # Append a new player dictionary if the player index exceeds the current list length
        if len(gameData[curTeam]["players"]) <= i:
            gameData[curTeam]["players"].append({"name": name, "position": position.upper(), "shots": {"Finishing": [0, 0], "Inside Shot": [0, 0], "Mid-Range": [0, 0], "3-Pointer": [0, 0]}, "driving": [0, 0]})
        else:
            # Update existing player information
            gameData[curTeam]["players"][i]["name"] = name
            gameData[curTeam]["players"][i]["position"] = position.upper()
            gameData[curTeam]["players"][i]["shots"] = {"Finishing": [0, 0], "Inside Shot": [0, 0], "Mid-Range": [0, 0], "3-Pointer": [0, 0]}
            gameData[curTeam]["players"][i]["driving"] = [0, 0]

        # Switch the team if the next player is "Total" and the current team is "homeTeam"
        if infoListPlayers[i + 1].text == "Total" and curTeam == "awayTeam":
            curTeam = "homeTeam"

    



In [145]:
playbyText = soup.find_all("div",id="Boxscore")[1].text
gameArr = playbyText.split("\n")
tipOff = gameArr[10]

In [146]:
x = soup.find_all("div",id="Boxscore")[1]
teamEventArr = x.find_all("b")[3:] #shows time and Team for each play-by-play Event (lined up with gameStartArr)

#Lines up the events of gameEventsArr and teamEventArr
gameEventsArr = gameArr[10:] # play-by-play Events
for i in gameEventsArr:
  if i == "":
    gameEventsArr.remove(i)


In [147]:
#Changes all Exclamations to Periods
for i in range(len(gameEventsArr)):
  gameEventsArr[i] = gameEventsArr[i].replace("!",".")
gameEventsArr = gameEventsArr[:-1]


In [148]:
#Delete "Game Event","2nd Half","Overtime" and lines w/o ":" from gameEventsArr 
'''
gameEventsArr = [string for string in gameEventsArr if "Game Event" not in string]
gameEventsArr = [string for string in gameEventsArr if "2nd Half" not in string]
gameEventsArr = [string for string in gameEventsArr if "Overtime" not in string]
gameEventsArr = [string for string in gameEventsArr if ":" in string]
'''
gameEventsArr = [
    string for string in gameEventsArr
    if "Game Event" not in string
    and "2nd Half" not in string
    and "Overtime" not in string
    and ":" in string
]
gameEventsArr

['\xa0\xa019:55 - UH-Hilo: Rowe brings up the ball against pressure defense.   Rowe passes the ball to Carson in the paint. Foul.  Woods commits the foul on Carson.    ',
 '\xa0\xa019:48 - UH-Hilo: The ball is inbounded to Gao.   Gao shoots from downtown.  The shot misses.   Woods grabs the long rebound.    ',
 '\xa0\xa019:43 - Dominican Univ. of California: Fast break opportunity.   Merritt leads the fast break with Ford and Marshall.   Merritt lays it up.  The shot goes in.   Dominican Univ. of California 2 - 0.    ',
 '\xa0\xa019:37 - UH-Hilo: Moran brings up the ball against man-to-man defense.   Moran passes the ball to Liao in the paint. Liao shoots in the paint.  The shot misses.   Woods grabs the rebound.    ',
 '\xa0\xa019:28 - Dominican Univ. of California: Fast break opportunity.   Merritt leads the fast break with Marshall and Ford.   Merritt passes the ball to Marshall on the perimeter. Marshall shoots from beyond the arc.  The shot misses.   Foul.  Liao fouls Watson block

In [149]:
#Deletes time of play-by-play, Makes it easier to get the Offensive Team


for i in range(len(gameEventsArr)):
  dash = "-"
  index = 0
  while(dash != gameEventsArr[i][index]):
      index += 1
  gameEventsArr[i] = gameEventsArr[i][index + 2:]


In [150]:
#Determines whether a drive is successful or not
Driving = {
  "Fail":"drive ",
  "Success":"drives "
}

#Driving["Fail"] in event

In [151]:
#Adds all the drivers
drive_attempt = -1 #-1: No Drive, 0: Fail, 1: Success
for event in gameEventsArr:
  if Driving["Fail"] in event or Driving["Success"] in event:
    drive_attempt = 0
    if Driving["Success"] in event:
      drive_attempt = 1
    words = event.split()
    team = words[0][:-1] #team of driver
    if drive_attempt == 0:
      drive_index = words.index("drive")
      player_index = drive_index - 3
    else:
      drive_index = words.index("drives")
      player_index = drive_index - 1
    player_name = words[player_index] #driver name

    
    if team in gameData["homeTeam"]["name"]:
      team = "homeTeam"
    else:
      team = "awayTeam"
    for player in gameData[team]["players"]:
      if player["name"] == player_name:
        player["driving"][0] += drive_attempt
        player["driving"][1] += 1


    


In [152]:
#Delete Game Events with Non-And One Fouls (second free throws occur) and Charging Fouls
gameEventsArr = [string for string in gameEventsArr if "second free throw" not in string or "blocking out" in string]
gameEventsArr = [string for string in gameEventsArr if "charged with the foul" not in string]


In [153]:

shotTypes = {    
    "Inside Shot": ["shoots from the inside","shoots from the low post", "shoots in the paint", "shoots from inside the arc", "shoots from the block","tips it in","attempts to dunk it" , "lays it up", "goes for the dunk"],
    "Mid-Range": ["with a fadeaway jumper","shoots a jumper"],
    "3-Pointer": ["shoots from beyond the arc","shoots from well beyond the arc","shoots from the three point line", "shoots from deep" , "shoots from the corner","shoots from downtown"]
}

Finishing = {"attempts to dunk it" , "lays it up", "goes for the dunk"}

In [154]:
#Adds every FG into the gameData per Player
for event in gameEventsArr:
  for shot_type, shots in shotTypes.items():
    for shot in shots:
      shot_attempt = 0
      if shot in event:
        words = event.split()
        player_index = words.index(shot.split()[0]) - 1
        player_name = words[player_index]
        team = words[0][:-1]
        if "Slam dunk" in event or "shot goes in" in event or "tips it in" == shot:
          shot_attempt = 1
        
        #For Fast-breaks
        if shot in Finishing and "Breakaway" in event or "Fast break opportunity" in event and "slow it down" not in event:
          shot_type = "Finishing"

        #For Drives
        if shot in Finishing and "drives" in event and words[words.index("drives") - 1] == player_name:
          shot_type = "Finishing"

        #finds player and accumlates the shot in gameData
        if team in gameData["homeTeam"]["name"]:
          team = "homeTeam"
        else:
          team = "awayTeam"
        for player in gameData[team]["players"]:
          if player["name"] == player_name:
            player["shots"][shot_type][0] += shot_attempt
            player["shots"][shot_type][1] += 1
        
        
gameData


{'awayTeam': {'name': 'UH-Hilo Vulcans',
  'teamCode': '565',
  'players': [{'name': 'Gao',
    'position': 'SF',
    'shots': {'Finishing': [1, 1],
     'Inside Shot': [3, 3],
     'Mid-Range': [2, 6],
     '3-Pointer': [5, 10]},
    'driving': [1, 3]},
   {'name': 'Liao',
    'position': 'PF',
    'shots': {'Finishing': [0, 0],
     'Inside Shot': [6, 10],
     'Mid-Range': [0, 1],
     '3-Pointer': [0, 0]},
    'driving': [0, 0]},
   {'name': 'Carson',
    'position': 'C',
    'shots': {'Finishing': [0, 0],
     'Inside Shot': [8, 13],
     'Mid-Range': [0, 2],
     '3-Pointer': [0, 0]},
    'driving': [0, 4]},
   {'name': 'Moran',
    'position': 'SG',
    'shots': {'Finishing': [0, 0],
     'Inside Shot': [1, 1],
     'Mid-Range': [0, 2],
     '3-Pointer': [1, 9]},
    'driving': [1, 2]},
   {'name': 'Rowe',
    'position': 'PG',
    'shots': {'Finishing': [2, 2],
     'Inside Shot': [0, 0],
     'Mid-Range': [0, 1],
     '3-Pointer': [0, 0]},
    'driving': [0, 1]},
   {'name': '